# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f368451f820>
├── 'a' --> tensor([[-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892]])
└── 'x' --> <FastTreeValue 0x7f368451fcd0>
    └── 'c' --> tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                        [-0.9262, -2.1147,  0.4540, -0.6911],
                        [ 1.4394,  0.6997, -0.0961, -1.6735]])

In [4]:
t.a

tensor([[-1.7489, -0.4396,  0.5087],
        [-0.5844,  0.6968, -0.2892]])

In [5]:
%timeit t.a

63.7 ns ± 0.0317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f368451f820>
├── 'a' --> tensor([[-1.1976, -0.4982,  0.7542],
│                   [ 0.0303, -0.2662,  1.6067]])
└── 'x' --> <FastTreeValue 0x7f368451fcd0>
    └── 'c' --> tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                        [-0.9262, -2.1147,  0.4540, -0.6911],
                        [ 1.4394,  0.6997, -0.0961, -1.6735]])

In [7]:
%timeit t.a = new_value

67.8 ns ± 0.099 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892]]),
    x: Batch(
           c: tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                      [-0.9262, -2.1147,  0.4540, -0.6911],
                      [ 1.4394,  0.6997, -0.0961, -1.6735]]),
       ),
)

In [10]:
b.a

tensor([[-1.7489, -0.4396,  0.5087],
        [-0.5844,  0.6968, -0.2892]])

In [11]:
%timeit b.a

57.8 ns ± 0.0178 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0315, -0.1069, -1.8952],
               [-0.0654, -1.3211,  0.9218]]),
    x: Batch(
           c: tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                      [-0.9262, -2.1147,  0.4540, -0.6911],
                      [ 1.4394,  0.6997, -0.0961, -1.6735]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.448 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

843 ns ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 24.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 298 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f35cf7c1100>
├── 'a' --> tensor([[[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]],
│           
│                   [[-1.7489, -0.4396,  0.5087],
│                    [-0.5844,  0.6968, -0.2892]]])
└── 'x' --> <FastTreeValue 0x7f35cf7c1160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 99.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f35e0e324f0>
├── 'a' --> tensor([[-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892],
│                   [-1.7489, -0.4396,  0.5087],
│                   [-0.5844,  0.6968, -0.2892]])
└── 'x' --> <FastTreeValue 0x7f35cfdaeb80>
    └── 'c' --> tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                        [-0.9262, -2.1147,  0.4540, -0.6911],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.9 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]],
       
               [[-1.7489, -0.4396,  0.5087],
                [-0.5844,  0.6968, -0.2892]]]),
    x: Batch(
           c: tensor([[[-0.4322, -1.1533, -1.7097, -0.6729],
                       [-0.9262, -2.1147,  0.4540, -0.6911],
                       [ 1.4394,  0.6997, -0.0961, -1.6735]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892],
               [-1.7489, -0.4396,  0.5087],
               [-0.5844,  0.6968, -0.2892]]),
    x: Batch(
           c: tensor([[-0.4322, -1.1533, -1.7097, -0.6729],
                      [-0.9262, -2.1147,  0.4540, -0.6911],
                      [ 1.4394,  0.6997, -0.0961, -1.6735],
                      [-0.4322, -1.1533, -1.7097, -0.6729],
                      [-0.9262, -

In [28]:
%timeit Batch.cat(batches)

160 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
